# EDA - Fraud Detection

<p align="center">
<img src="../assets/illustration_eda.png" alt="drawing" width="800"/>
<p>


In [1]:
# prelude

import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
# import random

from datetime import datetime, timedelta



k_AssetsDir     = "../data/"
k_FileName      = "fraud_test.csv"
k_Gold          = 1.618     # gold number for ratio
k_Width         = 12
k_Height        = k_Width/k_Gold
k_random_state  = 42           
k_test_size     = 20/100        


In [5]:
# -----------------------------------------------------------------------------
filename_in = k_AssetsDir+k_FileName
df = pd.read_csv(filename_in)
df.columns = df.columns.str.lower()
df.head(3)

,unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0


In [3]:
# -----------------------------------------------------------------------------
def quick_View(df):
  summary_lst = []
  
  for col_name in df.columns:
    col_dtype               = df[col_name].dtype
    num_of_null             = df[col_name].isnull().sum()
    percent_of_null         = num_of_null/len(df)
    num_of_non_null         = df[col_name].notnull().sum()
    num_of_distinct_values  = df[col_name].nunique()
    
    if num_of_distinct_values <= 10:
        distinct_values_counts = df[col_name].value_counts().to_dict()
    else:
        top_10_values_counts    = df[col_name].value_counts().head(10).to_dict()
        distinct_values_counts  = {k: v for k, v in sorted(top_10_values_counts.items(), key=lambda item: item[1], reverse=True)}

    if col_dtype != "object":
       max_of_col = df[col_name].max()
       min_of_col = df[col_name].min()
       outlier_hi = df[col_name].mean() + 3*df[col_name].std()
       outlier_lo = df[col_name].mean() - 3*df[col_name].std()
    else:
       max_of_col = -1
       min_of_col =  1
       outlier_hi = -1
       outlier_lo =  1
    
    summary_lst.append({
      "name"                : col_name,
      "dtype"               : col_dtype,
      "# null"              : num_of_null,
      "% null"              : (100*percent_of_null).round(2),
      "# NOT null"          : num_of_non_null,
      "distinct val"        : num_of_distinct_values,
      "-3*sig"              : round(outlier_lo,2) ,
      "min"                 : round(min_of_col,2),
      "max"                 : round(max_of_col,2),
      "+3*sig"              : round(outlier_hi,2) ,
      "distinct val count"  : distinct_values_counts
    })
  
  tmp_df = pd.DataFrame(summary_lst)
  return tmp_df

In [4]:
tmp_df = quick_View(df)
display(tmp_df.sort_values(by="# null", ascending=False))                 

,name,dtype,# null,% null,# NOT null,distinct val,-3*sig,min,max,+3*sig,distinct val count
0,unnamed: 0,int64,0,0.0,555719,555719,-2.034082e+05,0.000000e+00,5.557180e+05,7.591262e+05,"{0: 1, 370474: 1, 370488: 1, 370487: 1, 370486..."
12,zip,int64,0,0.0,555719,912,-3.172322e+04,1.257000e+03,9.992100e+04,1.294085e+05,"{82514: 1589, 48088: 1518, 34112: 1495, 16114:..."
21,merch_long,float64,0,0.0,555719,551770,-1.314300e+02,-1.666700e+02,-6.695000e+01,-4.903000e+01,"{-95.275575: 3, -82.357941: 3, -86.365442: 3, ..."
20,merch_lat,float64,0,0.0,555719,546490,2.326000e+01,1.903000e+01,6.668000e+01,5.383000e+01,"{39.779565000000005: 4, 40.952544: 4, 40.48193..."
19,unix_time,int64,0,0.0,555719,544760,1.365076e+09,1.371817e+09,1.388534e+09,1.396282e+09,"{1387468942: 4, 1386957227: 4, 1387312599: 4, ..."
18,trans_num,object,0,0.0,555719,555719,1.000000e+00,1.000000e+00,-1.000000e+00,-1.000000e+00,"{'2da90c7d74bd46a0caf3777415b3ebd3': 1, '7b0f0..."
17,dob,object,0,0.0,555719,910,1.000000e+00,1.000000e+00,-1.000000e+00,-1.000000e+00,"{'1977-03-23': 2408, '1988-09-15': 1951, '1981..."
16,job,object,0,0.0,555719,478,1.000000e+00,1.000000e+00,-1.000000e+00,-1.000000e+00,"{'Film/video editor': 4119, 'Exhibition design..."
15,city_pop,int64,0,0.0,555719,835,-8.129508e+05,2.300000e+01,2.906700e+06,9.893946e+05,"{606: 2553, 1312922: 2222, 1595797: 2182, 241:..."
14,long,float64,0,0.0,555719,910,-1.314000e+02,-1.656700e+02,-6.795000e+01,-4.907000e+01,"{-108.8964: 1589, -82.9832: 1518, -81.7361: 14..."


### <span style="color:orange"><b>Comments :</b></span>

Le tableau ci-dessus montre que : 

* Il y a 23 features (0 à 22)
* La première est un index
* Aucune feature ne comporte de valeur nulle (voir les colonnes # null, % null et # NOT null)
* Pas de soucis dans l'index. Il y a 555_719 ligne avec des index différents
* Rien de très remarquable à part la très grande disparité entre les nombres de transactions frauduleuses (2145) et légitimes (553_574)
* Dans le dataset il y a donc **0.38%** de transactions frauduleuses ($\frac{2145}{(553574+2145)}$)
* Compte tenu du contexte (détection de fraudes à la carte bancaire) il est décidé de porter une attention particulière au taux de faux négatif. C'est le taux de transaction frauduleuses qui seront classées comme légitimes. On souhaite que cette valeur soit minimale. Il est très important d'en détecter un maximum même si certaines transactions légitimes sont classées comme frauduleuses.
* En plus du taux de faux négatif, suite à l'entrainement on sauvegardera sur le serveur mlflow tracking (https://fraud-202406-70e02a9739f2.herokuapp.com/) la matrice de confusion, la ROC curve etc.

### <span style="color:orange"><b>Conclusion :</b></span>
* **0.38%** : le taux de transactions frauduleuse dans le jeu de test
* Métrique principale : Taux de faux négatifs
